In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score,
                             precision_recall_curve, f1_score, roc_curve)
import xgboost as xgb
import lightgbm as lgb
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# ============================================================================
# PASO 1: CARGA Y PREPROCESAMIENTO INICIAL
# ============================================================================
print("=" * 70)
print("CARGANDO Y PREPARANDO DATOS...")
print("=" * 70)

df = pd.read_csv('/content/BrFlights2.csv', encoding='latin1', low_memory=False)
print(f"✓ Filas originales: {df.shape[0]:,}")

# Filtrado y selección de columnas
df = df[df['Situacao.Voo'] == 'Realizado']
cols_to_keep = ['Companhia.Aerea', 'Aeroporto.Origem', 'Aeroporto.Destino',
                'Partida.Prevista', 'Partida.Real']
df = df[cols_to_keep].dropna()

# Conversión de fechas y creación de target
df['Partida.Prevista'] = pd.to_datetime(df['Partida.Prevista'])
df['Partida.Real'] = pd.to_datetime(df['Partida.Real'])
df['delay_minutes'] = (df['Partida.Real'] - df['Partida.Prevista']).dt.total_seconds() / 60
df['target'] = np.where(df['delay_minutes'] > 15, 1, 0)

# Features básicas temporales
df['hora_partida'] = df['Partida.Prevista'].dt.hour
df['dia_semana'] = df['Partida.Prevista'].dt.dayofweek
df['mes'] = df['Partida.Prevista'].dt.month
df['dia_mes'] = df['Partida.Prevista'].dt.day

# Renombrar
df = df.rename(columns={
    'Companhia.Aerea': 'companhia',
    'Aeroporto.Origem': 'origem',
    'Aeroporto.Destino': 'destino'
})

print(f"✓ Filas después de limpieza: {df.shape[0]:,}")
print(f"✓ Desbalance: {df['target'].value_counts(normalize=True).to_dict()}")

# ============================================================================
# PASO 2: FEATURE ENGINEERING AVANZADO
# ============================================================================
print("\n" + "=" * 70)
print("CREANDO FEATURES AVANZADAS...")
print("=" * 70)

# 2.1 VARIABLES TEMPORALES MEJORADAS
df['es_fin_semana'] = df['dia_semana'].isin([5, 6]).astype(int)
df['es_lunes'] = (df['dia_semana'] == 0).astype(int)
df['es_viernes'] = (df['dia_semana'] == 4).astype(int)

# Horas pico (6-9am y 5-8pm)
df['es_hora_pico'] = df['hora_partida'].isin([6,7,8,9,17,18,19,20]).astype(int)
df['es_madrugada'] = df['hora_partida'].isin([0,1,2,3,4,5]).astype(int)

# Temporadas (aproximación Brasil: verano dic-feb, invierno jun-ago)
df['temporada_alta'] = df['mes'].isin([1,2,7,12]).astype(int)

# Fin/inicio de mes (más tráfico corporativo)
df['fin_inicio_mes'] = df['dia_mes'].isin(list(range(1,6)) + list(range(26,32))).astype(int)

# 2.2 ESTADÍSTICAS HISTÓRICAS (Feature Engineering clave)
print("  → Calculando estadísticas históricas por aerolínea...")
airline_stats = df.groupby('companhia').agg({
    'target': ['mean', 'count'],
    'delay_minutes': ['mean', 'std']
}).reset_index()
airline_stats.columns = ['companhia', 'airline_delay_rate', 'airline_flights',
                         'airline_avg_delay', 'airline_std_delay']
df = df.merge(airline_stats, on='companhia', how='left')

print("  → Calculando estadísticas por ruta...")
route_stats = df.groupby(['origem', 'destino']).agg({
    'target': 'mean',
    'delay_minutes': 'mean'
}).reset_index()
route_stats.columns = ['origem', 'destino', 'route_delay_rate', 'route_avg_delay']
df = df.merge(route_stats, on=['origem', 'destino'], how='left')

print("  → Calculando estadísticas por aeropuerto origen...")
origin_stats = df.groupby('origem').agg({
    'target': 'mean',
    'delay_minutes': 'count'
}).reset_index()
origin_stats.columns = ['origem', 'origin_delay_rate', 'origin_flights']
df = df.merge(origin_stats, on='origem', how='left')

print("  → Calculando estadísticas por hora del día...")
hour_stats = df.groupby('hora_partida')['target'].mean().reset_index()
hour_stats.columns = ['hora_partida', 'hour_delay_rate']
df = df.merge(hour_stats, on='hora_partida', how='left')

# 2.3 INTERACCIONES CLAVE
print("  → Creando variables de interacción...")
df['companhia_origen'] = df['companhia'] + '_' + df['origem']
df['hora_dia_semana'] = df['hora_partida'].astype(str) + '_' + df['dia_semana'].astype(str)

# 2.4 FEATURES DE VOLUMEN (normalizado)
df['airline_popularity'] = df['airline_flights'] / df['airline_flights'].max()
df['origin_traffic'] = df['origin_flights'] / df['origin_flights'].max()

print(f"✓ Total de features creadas: {df.shape[1]}")

# ============================================================================
# PASO 3: CODIFICACIÓN Y PREPARACIÓN FINAL
# ============================================================================
print("\n" + "=" * 70)
print("CODIFICANDO VARIABLES CATEGÓRICAS...")
print("=" * 70)

# Encoders
le_companhia = LabelEncoder()
le_origem = LabelEncoder()
le_destino = LabelEncoder()
le_comp_orig = LabelEncoder()
le_hour_day = LabelEncoder()

df['companhia_encoded'] = le_companhia.fit_transform(df['companhia'].astype(str))
df['origem_encoded'] = le_origem.fit_transform(df['origem'].astype(str))
df['destino_encoded'] = le_destino.fit_transform(df['destino'].astype(str))
df['companhia_origen_encoded'] = le_comp_orig.fit_transform(df['companhia_origen'].astype(str))
df['hora_dia_encoded'] = le_hour_day.fit_transform(df['hora_dia_semana'].astype(str))

# Definir features finales
features = [
    # Básicas codificadas
    'companhia_encoded', 'origem_encoded', 'destino_encoded',
    # Temporales
    'hora_partida', 'dia_semana', 'mes', 'dia_mes',
    'es_fin_semana', 'es_lunes', 'es_viernes', 'es_hora_pico',
    'es_madrugada', 'temporada_alta', 'fin_inicio_mes',
    # Estadísticas históricas
    'airline_delay_rate', 'airline_avg_delay', 'airline_std_delay',
    'route_delay_rate', 'route_avg_delay',
    'origin_delay_rate', 'hour_delay_rate',
    # Interacciones
    'companhia_origen_encoded', 'hora_dia_encoded',
    # Volumen
    'airline_popularity', 'origin_traffic'
]

X = df[features].fillna(0)  # Rellenar NaN si existen
y = df['target']

# Split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✓ Training set: {X_train.shape[0]:,} samples")
print(f"✓ Test set: {X_test.shape[0]:,} samples")
print(f"✓ Total features: {X_train.shape[1]}")

# ============================================================================
# PASO 4: ENTRENAMIENTO DE MÚLTIPLES MODELOS
# ============================================================================
print("\n" + "=" * 70)
print("ENTRENANDO Y COMPARANDO MODELOS...")
print("=" * 70)

# Configuración de validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Diccionario para guardar resultados
results = {}

# -----------------------------
# MODELO 1: Random Forest Optimizado
# -----------------------------
print("\n[1/4] Random Forest con búsqueda de hiperparámetros...")
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', 'balanced_subsample']
}

rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_search = RandomizedSearchCV(
    rf, rf_params, n_iter=20, cv=cv, scoring='f1',
    random_state=42, n_jobs=-1, verbose=0
)
rf_search.fit(X_train, y_train)
results['Random Forest'] = rf_search.best_estimator_
print(f"  ✓ Mejor F1 (CV): {rf_search.best_score_:.4f}")
print(f"  ✓ Mejores parámetros: {rf_search.best_params_}")

# -----------------------------
# MODELO 2: XGBoost
# -----------------------------
print("\n[2/4] XGBoost con búsqueda de hiperparámetros...")
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
xgb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'scale_pos_weight': [scale_pos_weight]
}

xgb_model = xgb.XGBClassifier(random_state=42, eval_metric='logloss', n_jobs=-1)
xgb_search = RandomizedSearchCV(
    xgb_model, xgb_params, n_iter=20, cv=cv, scoring='f1',
    random_state=42, n_jobs=-1, verbose=0
)
xgb_search.fit(X_train, y_train)
results['XGBoost'] = xgb_search.best_estimator_
print(f"  ✓ Mejor F1 (CV): {xgb_search.best_score_:.4f}")
print(f"  ✓ Mejores parámetros: {xgb_search.best_params_}")

# -----------------------------
# MODELO 3: LightGBM
# -----------------------------
print("\n[3/4] LightGBM con búsqueda de hiperparámetros...")
lgb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, -1],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 50, 70],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'class_weight': ['balanced']
}

lgb_model = lgb.LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1)
lgb_search = RandomizedSearchCV(
    lgb_model, lgb_params, n_iter=20, cv=cv, scoring='f1',
    random_state=42, n_jobs=-1, verbose=0
)
lgb_search.fit(X_train, y_train)
results['LightGBM'] = lgb_search.best_estimator_
print(f"  ✓ Mejor F1 (CV): {lgb_search.best_score_:.4f}")
print(f"  ✓ Mejores parámetros: {lgb_search.best_params_}")

# -----------------------------
# MODELO 4: Logistic Regression (Baseline)
# -----------------------------
print("\n[4/4] Logistic Regression (baseline rápido)...")
lr = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
lr.fit(X_train, y_train)
results['Logistic Regression'] = lr
print(f"  ✓ Modelo entrenado")

# ============================================================================
# PASO 5: EVALUACIÓN Y COMPARACIÓN
# ============================================================================
print("\n" + "=" * 70)
print("EVALUANDO MODELOS EN TEST SET...")
print("=" * 70)

comparison = []
for name, model in results.items():
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Métricas
    from sklearn.metrics import precision_score, recall_score
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    comparison.append({
        'Modelo': name,
        'F1-Score': f1,
        'Precision': precision,
        'Recall': recall,
        'ROC-AUC': roc_auc,
        'Accuracy': (y_pred == y_test).mean()
    })

    print(f"\n{name}:")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  ROC-AUC: {roc_auc:.4f}")
    print(f"  Accuracy: {(y_pred == y_test).mean():.4f}")

# Tabla comparativa
comparison_df = pd.DataFrame(comparison).sort_values('F1-Score', ascending=False)
print("\n" + "=" * 70)
print("RANKING DE MODELOS:")
print("=" * 70)
print(comparison_df.to_string(index=False))

# ============================================================================
# PASO 6: OPTIMIZACIÓN DE THRESHOLD (Mejor modelo)
# ============================================================================
best_model_name = comparison_df.iloc[0]['Modelo']
best_model = results[best_model_name]

print("\n" + "=" * 70)
print(f"OPTIMIZANDO THRESHOLD PARA: {best_model_name}")
print("=" * 70)

y_proba = best_model.predict_proba(X_test)[:, 1]
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba)

# Calcular F1 para cada threshold
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
best_threshold_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_idx]

print(f"✓ Threshold óptimo: {best_threshold:.4f}")
print(f"✓ F1-Score óptimo: {f1_scores[best_threshold_idx]:.4f}")

# Predicciones con threshold optimizado
y_pred_optimized = (y_proba >= best_threshold).astype(int)

print("\n--- COMPARACIÓN: Default (0.5) vs Optimizado ---")
print("\nThreshold 0.5:")
print(classification_report(y_test, best_model.predict(X_test), digits=4))

print(f"\nThreshold {best_threshold:.4f}:")
print(classification_report(y_test, y_pred_optimized, digits=4))

# ============================================================================
# PASO 7: ANÁLISIS DETALLADO DEL MEJOR MODELO
# ============================================================================
print("\n" + "=" * 70)
print("ANÁLISIS DETALLADO Y FEATURE IMPORTANCE")
print("=" * 70)

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_optimized)
print("\nMatriz de Confusión (threshold optimizado):")
print(cm)
print(f"\nVerdaderos Negativos: {cm[0,0]:,}")
print(f"Falsos Positivos: {cm[0,1]:,}")
print(f"Falsos Negativos: {cm[1,0]:,}")
print(f"Verdaderos Positivos: {cm[1,1]:,}")

# Feature Importance (si el modelo lo soporta)
if hasattr(best_model, 'feature_importances_'):
    importances = pd.DataFrame({
        'feature': features,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nTop 15 Features Más Importantes:")
    print(importances.head(15).to_string(index=False))

# ============================================================================
# PASO 8: GUARDAR MODELO Y ARTEFACTOS
# ============================================================================
print("\n" + "=" * 70)
print("GUARDANDO MODELO Y ARTEFACTOS...")
print("=" * 70)

artifacts = {
    'model': best_model,
    'threshold': best_threshold,
    'le_companhia': le_companhia,
    'le_origem': le_origem,
    'le_destino': le_destino,
    'le_comp_orig': le_comp_orig,
    'le_hour_day': le_hour_day,
    'features': features,
    'airline_stats': airline_stats,
    'route_stats': route_stats,
    'origin_stats': origin_stats,
    'hour_stats': hour_stats,
    'model_name': best_model_name
}

joblib.dump(artifacts, 'flight_model_optimized_v2.joblib')
print("✓ Modelo guardado en: flight_model_optimized_v2.joblib")

# Guardar comparación
comparison_df.to_csv('model_comparison_results.csv', index=False)
print("✓ Comparación guardada en: model_comparison_results.csv")

print("\n" + "=" * 70)
print("¡PROCESO COMPLETADO EXITOSAMENTE!")
print("=" * 70)
print(f"\nModelo final: {best_model_name}")
print(f"F1-Score: {comparison_df.iloc[0]['F1-Score']:.4f}")
print(f"Recall: {comparison_df.iloc[0]['Recall']:.4f}")
print(f"Precision: {comparison_df.iloc[0]['Precision']:.4f}")
print(f"Threshold: {best_threshold:.4f}")

CARGANDO Y PREPARANDO DATOS...
✓ Filas originales: 31,437
✓ Filas después de limpieza: 24,556
✓ Desbalance: {0: 0.8636585763153608, 1: 0.1363414236846392}

CREANDO FEATURES AVANZADAS...
  → Calculando estadísticas históricas por aerolínea...
  → Calculando estadísticas por ruta...
  → Calculando estadísticas por aeropuerto origen...
  → Calculando estadísticas por hora del día...
  → Creando variables de interacción...
✓ Total de features creadas: 31

CODIFICANDO VARIABLES CATEGÓRICAS...
✓ Training set: 19,644 samples
✓ Test set: 4,912 samples
✓ Total features: 25

ENTRENANDO Y COMPARANDO MODELOS...

[1/4] Random Forest con búsqueda de hiperparámetros...
  ✓ Mejor F1 (CV): 0.4505
  ✓ Mejores parámetros: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 15, 'class_weight': 'balanced_subsample'}

[2/4] XGBoost con búsqueda de hiperparámetros...
  ✓ Mejor F1 (CV): 0.4335
  ✓ Mejores parámetros: {'subsample': 0.7, 'scale_pos_weight': np.float64(6.3353248693

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


  ✓ Modelo entrenado

EVALUANDO MODELOS EN TEST SET...

Random Forest:
  F1-Score: 0.4541
  Precision: 0.4148
  Recall: 0.5015
  ROC-AUC: 0.7938
  Accuracy: 0.8355

XGBoost:
  F1-Score: 0.4627
  Precision: 0.3599
  Recall: 0.6478
  ROC-AUC: 0.8059
  Accuracy: 0.7948

LightGBM:
  F1-Score: 0.4530
  Precision: 0.3611
  Recall: 0.6075
  ROC-AUC: 0.8000
  Accuracy: 0.7999

Logistic Regression:
  F1-Score: 0.3763
  Precision: 0.2611
  Recall: 0.6731
  ROC-AUC: 0.7537
  Accuracy: 0.6956

RANKING DE MODELOS:
             Modelo  F1-Score  Precision   Recall  ROC-AUC  Accuracy
            XGBoost  0.462687   0.359867 0.647761 0.805910  0.794788
      Random Forest  0.454054   0.414815 0.501493 0.793843  0.835505
           LightGBM  0.452977   0.361136 0.607463 0.799958  0.799878
Logistic Regression  0.376304   0.261146 0.673134 0.753720  0.695643

OPTIMIZANDO THRESHOLD PARA: XGBoost
✓ Threshold óptimo: 0.5314
✓ F1-Score óptimo: 0.4850

--- COMPARACIÓN: Default (0.5) vs Optimizado ---

Thresho